In [14]:
#!/usr/bin/env python3

import unittest

from archemist.stations.mt_synthesis_station.state import (
    MTSynthesisStation,
    MTSynthHeatStirOp,
    MTSynthSampleOp,
    MTSynthStopReactionOp,
    MTSynthTimedOpenReactionValveOp,
    MTSynthOpenReactionValveOp,
    MTSynthCloseReactionValveOp
)

from archemist.core.state.material import Liquid

from archemist.stations.mt_synthesis_station.handler import APCMTSynthesisStationRosHandler
from archemist.core.state.batch import Batch
from archemist.core.state.lot import Lot
from archemist.core.util.enums import StationState, OpOutcome
from archemist.core.state.station_op_result import ProcessOpResult, MaterialOpResult
from archemist.core.persistence.models_proxy import ModelProxy, ListProxy, DictProxy
from archemist.stations.apc_weighing_station.handler import SimAPCWeighingStationHandler

from archemist.core.state.robot_op import RobotTaskOp, RobotWaitOp
from archemist.core.state.batch import Batch
from archemist.core.state.lot import Lot
from archemist.core.util.enums import OpOutcome, ProcessStatus

from datetime import datetime

import time

from mongoengine import connect

print("importing done")


class APCSynthesisStationHandlerTest(unittest.TestCase):
    def setUp(self):
        self._db_name = 'archemist_test'
        self._client = connect(db=self._db_name, host='mongodb://localhost:27017', alias='archemist_state')

        self.station_doc = {
            'type': 'MTSynthesisStation',
            'id': 21,
            'location': {'coordinates': [1,7], 'descriptor': "MTSynthesisStation"},
            'total_lot_capacity': 1,
            'handler': 'SimStationOpHandler',
            'properties': 
            {
                'num_sampling_vials': 12
            },
            'materials': None
        }

    def test_handler(self):
        station = MTSynthesisStation.from_dict(self.station_doc)
        self.assertIsNotNone(station)
        self.assertEqual(station.state, StationState.INACTIVE)
            # construct lot
        batch = Batch.from_args(1)
        lot = Lot.from_args([batch])

        # add lot to station
        station.add_lot(lot)

        # construct handler
        handler = APCMTSynthesisStationRosHandler(station)
        # initialise handler
        self.assertTrue(handler.initialise())

        # test IKAHeatStirBatchOp
        # construct op
        sample = lot.batches[0].samples[0]
        # t_op = MTSynthHeatStirOp.from_args(target_sample=sample, target_temperature=20, target_stirring_speed=600,wait_duration=5,time_unit="minute", stir_duration=23)
        # t_op = MTSynthSampleOp.from_args(target_sample=sample, target_temperature=23, target_stirring_speed=200, dilution=90)
        # t_op = MTSynthStopReactionOp.from_args()
        # t_op = MTSynthTimedOpenReactionValveOp.from_args(duration=3,time_unit="second")
        # t_op = MTSynthOpenReactionValveOp.from_args()
        t_op = MTSynthCloseReactionValveOp.from_args()


        self.assertIsNotNone(t_op.object_id)

        station.add_station_op(t_op)
        station.update_assigned_op()
        self.assertEqual(station.assigned_op, t_op)


        # execute op
        handler._seq_id = 8
        handler.execute_op()

        # wait for results
        handler.is_op_execution_complete()

        self.assertEqual(handler.is_op_execution_complete(), False)

        while True:
            if handler.is_op_execution_complete():
                print("got the completion msg, moving on")
                break
            else:
                print("didn't get the completion msg, hence trying")
            time.sleep(2)

        self.assertEqual(handler.is_op_execution_complete(), True)



        outcome, op_results = handler.get_op_result()


        self.assertEqual(outcome, OpOutcome.SUCCEEDED)
        # self.assertIsInstance(op_results[0], ProcessOpResult)
        self.assertEqual(op_results, None)




importing done


In [15]:
# Create a TestSuite
# Balance_suite = unittest.TestLoader().loadTestsFromTestCase(APCWeighingStationHandlerTest)
optimax_suite = unittest.TestLoader().loadTestsFromTestCase(APCSynthesisStationHandlerTest)

# Run the tests
unittest.TextTestRunner().run(optimax_suite)


[Batch-65ba849f2a2156e2b7c85460]: (MTSynthesisStation_21) stamp is added.
[MTSynthesisStation_21]: Lot-65ba849f2a2156e2b7c85462 is added for processing at slot 0
[MTSynthesisStation_21]: <archemist.stations.mt_synthesis_station.state.MTSynthCloseReactionValveOp object at 0x7f20300690d0> is added to queued_op list
[INFO] [1706722465.247483]: Base valve close
didn't get the completion msg, hence trying
didn't get the completion msg, hence trying
didn't get the completion msg, hence trying
didn't get the completion msg, hence trying
didn't get the completion msg, hence trying


.

got the completion msg, moving on



----------------------------------------------------------------------
Ran 1 test in 12.118s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>